In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
import os
os.chdir("/home/romain/PycharmProjects/scVI-harmo/")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scvi

%matplotlib inline

In [106]:
%%bash 
mkdir data
curl https://ndownloader.figshare.com/files/18789473?private_link=6e12f5f56bb46842f5ee --output data/pbmc3k.h5ad 
curl https://raw.githubusercontent.com/normjam/ppcs/master/genes_1000_pbmc3k.csv --output data/genes_1000_pbmc3k.csv
curl https://raw.githubusercontent.com/normjam/ppcs/master/pbmc3k_generated_sctransform.h5 --output data/pbmc3k_generated_sctransform.h5
curl https://raw.githubusercontent.com/normjam/ppcs/master/pbmc3k_1kgenes_glmpca10_poi_samples.npz --output data/pbmc3k_1kgenes_glmpca10_poi_samples.npz
curl https://raw.githubusercontent.com/normjam/ppcs/master/pbmc3k_1kgenes_glmpca10_nb_samples.npz --output data/pbmc3k_1kgenes_glmpca10_nb_samples.npz

mkdir: cannot create directory ‘data’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.0M  100 11.0M    0     0  2207k      0  0:00:05  0:00:05 --:--:-- 2877k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11071  100 11071    0     0  45862      0 --:--:-- --:--:-- --:--:-- 45937
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.4M  100 20.4M    0     0  13.6M      0  0:00:01  0:00:01 --:--:-- 13.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.0M  100 15.0M    0     0  11.4M      0  0:00:01  0:00:01 --:--:-- 11.4M
 

In [4]:
import anndata
anndataset = anndata.read("data/pbmc3k.h5ad")

In [5]:
from scvi.dataset import AnnDatasetFromAnnData
dataset = AnnDatasetFromAnnData(ad = anndataset)

[2019-11-20 10:22:27,487] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-11-20 10:22:27,633] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-11-20 10:22:27,635] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-11-20 10:22:27,704] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-11-20 10:22:27,779] INFO - scvi.dataset.dataset | Downsampled from 2695 to 2695 cells


In [6]:
select_genes = np.array([x.split(",")[1].upper() for x in np.loadtxt("data/genes_1000_pbmc3k.csv", dtype=np.str)])
dataset.reorder_genes(select_genes, drop_omitted_genes=True)
assert(np.all(select_genes == dataset.gene_names))

[2019-11-20 10:22:27,880] INFO - scvi.dataset.dataset | Computing the library size for the new data


In [7]:
# Model training
from scvi.models import VAE
from scvi.inference import UnsupervisedTrainer

n_epochs = 200
warmup = 100
n_latent = 10
dict_post = {}

# ZINB scVI
for reconstruction_loss in ["zinb", "nb", "poisson"]:
    model = VAE(n_input=dataset.nb_genes, n_latent=n_latent, n_batch=0, reconstruction_loss=reconstruction_loss)
    trainer = UnsupervisedTrainer(
        model=model, 
        gene_dataset=dataset, 
        n_epochs_kl_warmup=warmup,
        frequency=1,
        train_size=1.0,
    )    
    trainer.train(n_epochs=n_epochs, lr=1e-2)
    dict_post[reconstruction_loss] = trainer.create_posterior(trainer.model, dataset, 
                                                              indices=np.arange(len(dataset))).sequential()


# PPC for the entire dataset

In [101]:
from scvi.inference import PosteriorPredictiveCheck

In [102]:
ppc = PosteriorPredictiveCheck(dict_post, n_samples=50)
# add factor analysis
ppc.store_fa_samples(n_components=30, normalization="log")
ppc.store_fa_samples(n_components=30, normalization="log_rate")
ppc.store_fa_samples(n_components=30, normalization="rate")

In [103]:
ppc.coeff_of_variation(cell_wise=True)
ppc.coeff_of_variation(cell_wise=False)
ppc.mean(cell_wise=False)
ppc.variance(cell_wise=False)
ppc.dropout_ratio()
ppc.gene_gene_correlation(gene_indices=np.arange(1000))
ppc.calibration_error([0, 100])
ppc.mean_squared_log_error()
ppc.median_absolute_error()

/home/romain/PycharmProjects/scVI-harmo/scvi/inference/ppc.py:59: RuntimeWarning: invalid value encountered in true_divide
  np.std(samples, axis=axis) / np.mean(samples, axis=axis), axis=-1
/home/romain/PycharmProjects/scVI-harmo/scvi/inference/ppc.py:65: RuntimeWarning: invalid value encountered in true_divide
  self.raw_counts, axis=axis


In [104]:
import pickle
# open a file, where you ant to store the data
file = open('results_scVI_and_FA.pickle', 'wb')

# dump information to that file
pickle.dump(ppc.metrics, file)

# close the file
file.close()

In [105]:
ppc = PosteriorPredictiveCheck(dict_post, n_samples=50)

In [107]:
import h5py
import numpy as np 

scTransform = h5py.File("data/pbmc3k_generated_sctransform.h5", 'r')  
scTransform = np.array(scTransform['x'])
scTransform = scTransform.swapaxes(0, 2)
ppc.store_external_samples(scTransform, "sctransform")

glm_pca_poisson = np.load("data/pbmc3k_1kgenes_glmpca10_poi_samples.npz")
glm_pca_poisson = list(glm_pca_poisson.values())[0].astype(np.float64)
ppc.store_external_samples(glm_pca_poisson, "glm_pca_poisson")

glm_pca_nb = np.load("data/pbmc3k_1kgenes_glmpca10_nb_samples.npz")
glm_pca_nb = list(glm_pca_nb.values())[0].astype(np.float64)
ppc.store_external_samples(glm_pca_nb, "glm_pca_nb")

In [108]:
ppc.coeff_of_variation(cell_wise=True)
ppc.coeff_of_variation(cell_wise=False)
ppc.mean(cell_wise=False)
ppc.variance(cell_wise=False)
ppc.dropout_ratio()
ppc.gene_gene_correlation(gene_indices=np.arange(1000))
ppc.calibration_error([0, 100])
ppc.mean_squared_log_error()
ppc.median_absolute_error()

/home/romain/PycharmProjects/scVI-harmo/scvi/inference/ppc.py:59: RuntimeWarning: invalid value encountered in true_divide
  np.std(samples, axis=axis) / np.mean(samples, axis=axis), axis=-1
/home/romain/PycharmProjects/scVI-harmo/scvi/inference/ppc.py:59: RuntimeWarning: Mean of empty slice
  np.std(samples, axis=axis) / np.mean(samples, axis=axis), axis=-1
/home/romain/PycharmProjects/scVI-harmo/scvi/inference/ppc.py:65: RuntimeWarning: invalid value encountered in true_divide
  self.raw_counts, axis=axis


In [109]:
import pickle
# open a file, where you ant to store the data
file = open('results_scVI_and_others.pickle', 'wb')

# dump information to that file
pickle.dump(ppc.metrics, file)

# close the file
file.close()